In [1]:
import string
import itertools
from collections import Counter
import nltk
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
import pandas
stopwords = ["a","able","about","above","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","after","afterwards","again","against","ah","all","almost","alone","along","already","also","although","always","am","among","amongst","an","and","announce","another","any","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","are","aren","arent","arise","around","as","aside","ask","asking","at","auth","available","away","awfully","b","back","be","became","because","become","becomes","becoming","been","before","beforehand","begin","beginning","beginnings","begins","behind","being","believe","below","beside","besides","between","beyond","biol","both","brief","briefly","but","by","c","ca","came","can","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","could","couldnt","d","date","did","didn't","different","do","does","doesn't","doing","done","don't","down","downwards","due","during","e","each","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","et-al","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","few","ff","fifth","first","five","fix","followed","following","follows","for","former","formerly","forth","found","four","from","further","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","had","happens","hardly","has","hasn't","have","haven't","having","he","hed","hence","her","here","hereafter","hereby","herein","heres","hereupon","hers","herself","hes","hi","hid","him","himself","his","hither","home","how","howbeit","however","hundred","i","id","ie","if","i'll","im","immediate","immediately","importance","important","in","inc","indeed","index","information","instead","into","invention","inward","is","isn't","it","itd","it'll","its","itself","i've","j","just","k","keep	keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","m","made","mainly","make","makes","many","may","maybe","me","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","more","moreover","most","mostly","mr","mrs","much","mug","must","my","myself","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine","ninety","no","nobody","non","none","nonetheless","noone","nor","normally","nos","not","noted","nothing","now","nowhere","o","obtain","obtained","obviously","of","off","often","oh","ok","okay","old","omitted","on","once","one","ones","only","onto","or","ord","other","others","otherwise","ought","our","ours","ourselves","out","outside","over","overall","owing","own","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","re","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","s","said","same","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","she","shed","she'll","shes","should","shouldn't","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","so","some","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","stop","strongly","sub","substantially","successfully","such","sufficiently","suggest","sup","sure	t","take","taken","taking","tell","tends","th","than","thank","thanks","thanx","that","that'll","thats","that've","the","their","theirs","them","themselves","then","thence","there","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","these","they","theyd","they'll","theyre","they've","think","this","those","thou","though","thoughh","thousand","throug","through","throughout","thru","thus","til","tip","to","together","too","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","under","unfortunately","unless","unlike","unlikely","until","unto","up","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","very","via","viz","vol","vols","vs","w","want","wants","was","wasnt","way","we","wed","welcome","we'll","went","were","werent","we've","what","whatever","what'll","whats","when","whence","whenever","where","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","which","while","whim","whither","who","whod","whoever","whole","who'll","whom","whomever","whos","whose","why","widely","willing","wish","with","within","without","wont","words","world","would","wouldnt","www","x","y","yes","yet","you","youd","you'll","your","youre","yours","yourself","yourselves","you've","z","zero"]
#stopwords = stopwords + ["player","players"] # used to build a bi-gram graph without players in it.
remove_punct = str.maketrans({key: None for key in string.punctuation})

In [2]:
d = pandas.read_csv("/Users/michaelshea/Documents/dm_survey_data/dm_survey_clean.csv", encoding = 'iso-8859-1')
tools = [x for x in list(d["Top Three Tools"]) if x == x] # Remove empty responses
tips = [x for x in list(d["Favorite Trick"]) if x == x]
tips = list(set(tips))
tools = list(set(tools))
print(len(tools))
print(len(tips))

4164
4046


In [16]:
my_tool_list = []
unprocessed_count = 0
stop_words = ["the","and"]
for response in tools:
    try:
        response = response.replace("\n",",") # turn line breaks into commas
    except:
        meh = 1
    tool_set = response.split(",")
    for tool in tool_set:
        # Bunch of replacements to normalize on common tools
        tool = tool.strip().lower()
        tool = " ".join([x for x in tool.split() if x not in stop_words])
        tool = tool.replace(".","")
        tool = tool.replace("'","")
        tool = tool.replace("dmg","dungeon masters guide")
        tool = tool.replace("dungeon master guide","dungeon masters guide")
        tool = tool.replace("mm","monster manual")
        tool = tool.replace("monster manuel","monster manual")
        tool = tool.replace("monsters manual","monster manual")
        tool = tool.replace("dm guide","dungeon masters guide")
        tool = tool.replace("dms guide","dungeon masters guide")
        tool = tool.replace("core books","core rulebooks")
        tool = tool.replace("phb","players handbook")
        tool = tool.replace("phb","players handbook")
        tool = tool.replace("roll 20","roll20")
        tool = tool.replace("one note","onenote")
        tool = tool.replace("donjonbinsh","donjon")
        tool = tool.replace("microsoft onenote","onenote")
        tool = tool.replace("koboldclub","kobold fight club")
        tool = tool.replace("player handbook","players handbook")
        tool = tool.replace("core rule books","core rulebooks")
#        if "kobold" in tool.split() and "fight" in tool.split(): tool = "kobold fight club"
        if "donjon" in tool: tool = "donjon"
        if "kobold" in tool and "fight" in tool: tool = "kobold fight club"
        if "dungeon masters guide" in tool: tool = "dungeon masters guide"
        if "monster manual" in tool: tool = "monster manual"
        if "players handbook" in tool: tool = "players handbook"
        if tool != "": # get rid of blank entries
            my_tool_list.append(tool)
c = Counter(my_tool_list)
p = pandas.DataFrame(list(c.most_common(5000)))
p.columns = ["Tool","Freq"]
p = p[p["Freq"] >= 2]
p.to_csv("/Users/michaelshea/Documents/dm_survey_data/2016_dm_survey_tools.csv")
p.head(30)

,Tool,Freq
0,kobold fight club,569
1,dungeon masters guide,548
2,monster manual,471
3,donjon,449
4,roll20,374
5,onenote,268
6,reddit,209
7,players handbook,169
8,google docs,135
9,paper,104


In [4]:
def get_word_count_df(list_of_texts):
    wordnet_lemmatizer = WordNetLemmatizer()
    words = [x.split() for x in list_of_texts] # Turn each response into a list of words
    words = list(itertools.chain.from_iterable(words)) # Flatten the words into a single list of all words
    words = [wordnet_lemmatizer.lemmatize(x.lower().translate(remove_punct)) for x in words 
             if x.lower().translate(remove_punct) not in stopwords 
             and x.lower().translate(remove_punct) != ""] # Remove stopwords, punctuation and lower case everything.
    c = Counter(words) # build a counter to count up word frequency
    pd = pandas.DataFrame(c.most_common(500000)) # Give us the top 500 words and turn into a Pandas dataframe
    pd.columns = ["word","Freq"] # Set the columns
    return(pd)

In [5]:
wc = get_word_count_df(tips).head(50)
for x in wc.iterrows():
    print(x[1]["word"] + " (" + str(x[1]["Freq"]) + "), ", end="")

player (2044), story (522), character (461), game (425), npc (410), dont (350), fun (278), thing (271), session (254), keep (244), time (233), encounter (217), will (205), pc (199), combat (198), play (175), idea (173), good (163), sure (153), adventure (148), rule (142), feel (138), roll (137), lot (135), party (131), making (130), action (129), plan (127), plot (127), dm (123), work (120), campaign (119), great (112), improv (110), going (101), create (99), group (93), improvise (93), choice (90), help (89), trick (86), monster (85), prep (84), find (83), build (83), letting (80), music (80), happen (80), table (79), prepare (79), 

In [6]:
def get_bigrams_trigrams_df(list_of_texts):
    wordnet_lemmatizer = WordNetLemmatizer()
    word_sets = [x.lower().translate(remove_punct).split() for x in list_of_texts]
    total_word_sets = []
    for word_set in word_sets:
        new_word_set = []
        for word in word_set:
            if word not in stopwords:
                new_word_set.append(wordnet_lemmatizer.lemmatize(word))
        total_word_sets.append(new_word_set)
    bigrams = list(itertools.chain.from_iterable(list(list(nltk.ngrams(x, 2) for x in total_word_sets))))
    bigrams_df = pandas.DataFrame(Counter(bigrams).most_common(100000), columns = ["trigram", "freq"])
    trigrams = list(itertools.chain.from_iterable(list(list(nltk.ngrams(x, 3) for x in total_word_sets))))
    trigrams_df = pandas.DataFrame(Counter(trigrams).most_common(100000), columns = ["trigram", "freq"])
    return(bigrams, bigrams_df, trigrams_df)

# Print out nice paragraphs of top 50 bigrams and trigrams
bigrams, bigram_frame, trigram_frame = get_bigrams_trigrams_df(tips)
tf = trigram_frame.head(50)

bf = bigram_frame.head(50)
for x in bf.iterrows():
    replacements = ["(",")",",","'"]
    print("".join([y for y in str(x[1]["trigram"]) if y not in replacements]) + 
          " (" + str(x[1]["freq"]) + "), ", end="")
    
print("\n")

for x in tf.iterrows():
    replacements = ["(",")",",","'"]
    print("".join([y for y in str(x[1]["trigram"]) if y not in replacements]) + 
          " (" + str(x[1]["freq"]) + "), ", end="")


player character (54), player will (51), letting player (49), listen player (42), player story (39), making sure (38), player feel (32), player game (32), player dont (31), sure player (30), player action (29), keep player (29), player idea (28), dont afraid (27), game player (27), player fun (27), allow player (26), player choice (26), story player (25), keep thing (25), combat encounter (25), role play (22), character story (21), knowing player (19), player decide (19), player agency (19), random encounter (18), player play (18), player find (18), thing player (17), rule cool (17), fun player (17), player thing (17), listening player (17), drive story (16), ahead time (16), magic item (16), player describe (15), plot twist (15), plot point (15), npc player (15), player going (15), memorable npc (14), role playing (14), player engaged (14), player control (14), plot hook (14), side quest (14), spend time (14), dm screen (13), 

keep thing moving (13), player drive story (11), making s

In [7]:
# Output survey tip word count CSV
tips_wc_pd = get_word_count_df(tips)
tips_wc_pd.to_csv("/Users/michaelshea/Documents/dm_survey_data/2016_dm_survey_tip_word_count.csv", index=False)

In [8]:
# Output bigrams spreadsheets both frequency rollup and a flat one for the graph.
bigrams, bigram_frame, trigram_frame = get_bigrams_trigrams_df(tools)
pandas.DataFrame(bigrams).to_csv("/Users/michaelshea/Documents/dm_survey_data/2016_dm_survey_tip_bigrams.csv", index=False)
bigram_frame = bigram_frame[bigram_frame["freq"] >= 2] # reduce the csv to only those that show up twice
bigram_frame.to_csv("/Users/michaelshea/Documents/dm_survey_data/2016_dm_survey_bigram_rollup.csv", index=False)
trigram_frame = trigram_frame[trigram_frame["freq"] >= 2] # reduce the csv to only those that show up twice
trigram_frame.to_csv("/Users/michaelshea/Documents/dm_survey_data/2016_dm_survey_trigram_rollup.csv", index=False)

In [9]:
stripped_tips = [x.replace("\n"," ") for x in tips]

In [10]:
with open("/Users/michaelshea/Documents/dm_survey_data/tips.txt","w") as f:
    for tip in stripped_tips:
        f.write(tip)
        f.write("\n\n")